In [2]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

#### Cargamos la data de clima

In [60]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_clima_pw = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_clima/PW.txt'
path_clima_camp8 = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_clima/Camp8.txt'
path_clima_fm = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_clima/FM 1.txt'
path_clima_shanusi = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_clima/Shanusi.txt'

path_clima_anteriores = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_clima/Plantilla_AGRI_Clima_WL.xlsx'



df_clima_pw = pd.read_csv(ruta_path + path_clima_pw, sep= '\t')
df_clima_camp8 = pd.read_csv(ruta_path + path_clima_camp8, sep= '\t')
df_clima_fm = pd.read_csv(ruta_path + path_clima_fm, sep= '\t')
df_clima_shanusi = pd.read_csv(ruta_path + path_clima_shanusi, sep= '\t')

df_clima_anteriores = pd.read_excel(ruta_path + path_clima_anteriores,sheet_name='Clima_WL')




In [30]:
df_clima_camp8.columns == df_clima_pw.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [31]:
# Filtrando columnas de interes
columns_clima = ['Unnamed: 0','Unnamed: 1','Temp','Hi','Low','Out','Solar']
# --
df_clima_pw_filter = df_clima_pw[columns_clima]
df_clima_camp8_filter = df_clima_camp8[columns_clima]
df_clima_fm_filter = df_clima_fm[columns_clima]
df_clima_shanusi_filter = df_clima_shanusi[columns_clima]

# Eliminando la segunda fila de cada dataframe
df_clima_pw_filter = df_clima_pw_filter.drop(0)
df_clima_camp8_filter = df_clima_camp8_filter.drop(0)
df_clima_fm_filter = df_clima_fm_filter.drop(0)
df_clima_shanusi_filter = df_clima_shanusi_filter.drop(0)

df_clima_camp8_filter['PLANTACION'] = 'Nuevo Horizonte'
df_clima_pw_filter['PLANTACION'] = 'Palmawasi'
df_clima_fm_filter['PLANTACION'] = 'Palmawasi'
df_clima_shanusi_filter['PLANTACION'] = 'Shanuzi'


#### Generando base total - Clima

In [41]:
df_clima_full = pd.concat([df_clima_camp8_filter,
                           df_clima_pw_filter,
                           df_clima_camp8_filter,
                           df_clima_shanusi_filter],
                           axis=0,
                           ignore_index=True)

print(df_clima_full.PLANTACION.unique())
# Renombrando variables
df_clima_full.columns = ['FECHA','Time','Temperatura','Temp Hi','Temp Low','Humedad','Radiacion','PLANTACION']

# Limpiando datos
df_clima_full['Temperatura'] = df_clima_full['Temperatura'].replace('---',np.nan)
df_clima_full['Temp Hi'] = df_clima_full['Temp Hi'].replace('---',np.nan)
df_clima_full['Temp Low'] = df_clima_full['Temp Low'].replace('---',np.nan)
df_clima_full['Humedad'] = df_clima_full['Humedad'].replace('---',np.nan)
df_clima_full['Radiacion'] = df_clima_full['Radiacion'].replace('---',np.nan)

# Transformado variables al formato correcto
# --
df_clima_full['FECHA'] =  pd.to_datetime(df_clima_full['FECHA'])
df_clima_full['Temperatura'] = df_clima_full['Temperatura'].astype('float')
df_clima_full['Temp Hi'] = df_clima_full['Temp Hi'].astype('float')
df_clima_full['Temp Low'] = df_clima_full['Temp Low'].astype('float')
df_clima_full['Humedad'] = df_clima_full['Humedad'].astype('float')
df_clima_full['Radiacion'] = df_clima_full['Radiacion'].astype('float')

df_clima_full.head()


['Nuevo Horizonte' 'Palmawasi' 'Shanuzi']


,FECHA,Time,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion,PLANTACION
0,01/01/24,0:15,24.3,24.3,24.2,95,0,Nuevo Horizonte
1,01/01/24,0:30,24.4,24.4,24.3,95,0,Nuevo Horizonte
2,01/01/24,0:45,24.2,24.4,24.2,94,0,Nuevo Horizonte
3,01/01/24,1:00,24.1,24.2,24.1,95,0,Nuevo Horizonte
4,01/01/24,1:15,24.0,24.1,23.9,95,0,Nuevo Horizonte


Añadiendo años anteriores

In [83]:
# Crear un diccionario de mapeo
mapping = {
    'Campamento 8 Shanusi': 'Nuevo Horizonte',
    'Fray Martin': 'Palmawasi',
    'Palmas Del Shanusi': 'Shanuzi',
    'Palmawasi': 'Palmawasi'
}
# Usar map y fillna
df_clima_anteriores['PLANTACION'] = df_clima_anteriores['Estacion'].map(mapping).fillna('Desconocido')

# Seleccionando variables de interes
df_clima_anteriores_sub = df_clima_anteriores[['Date','Time','Temp Out','Hi Temp','Low Temp','Out Hum','Solar Rad.','PLANTACION']]
df_clima_anteriores_sub.columns = ['FECHA','Time','Temperatura','Temp Hi','Temp Low','Humedad','Radiacion','PLANTACION']

df_clima_anteriores_sub.head()


,FECHA,Time,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion,PLANTACION
0,2010-01-01,00:30:00,20.9,21,20.8,100,0,Palmawasi
1,2010-01-01,01:00:00,20.8,20.9,20.8,100,0,Palmawasi
2,2010-01-01,01:30:00,20.8,20.8,20.7,100,0,Palmawasi
3,2010-01-01,02:00:00,20.8,20.8,20.8,100,0,Palmawasi
4,2010-01-01,02:30:00,20.8,20.9,20.8,100,0,Palmawasi


In [84]:
# Limpiando datos
df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].replace('---',np.nan)
df_clima_anteriores_sub['Temp Hi'] = df_clima_anteriores_sub['Temp Hi'].replace('---',np.nan)
df_clima_anteriores_sub['Temp Low'] = df_clima_anteriores_sub['Temp Low'].replace('---',np.nan)
df_clima_anteriores_sub['Humedad'] = df_clima_anteriores_sub['Humedad'].replace('---',np.nan)
df_clima_anteriores_sub['Radiacion'] = df_clima_anteriores_sub['Radiacion'].replace('---',np.nan)

df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].replace('--',np.nan)
df_clima_anteriores_sub['Temp Hi'] = df_clima_anteriores_sub['Temp Hi'].replace('--',np.nan)
df_clima_anteriores_sub['Temp Low'] = df_clima_anteriores_sub['Temp Low'].replace('--',np.nan)
df_clima_anteriores_sub['Humedad'] = df_clima_anteriores_sub['Humedad'].replace('--',np.nan)
df_clima_anteriores_sub['Radiacion'] = df_clima_anteriores_sub['Radiacion'].replace('--',np.nan)

df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].astype(str).str.replace(',','.')
df_clima_anteriores_sub['Temp Hi'] = df_clima_anteriores_sub['Temp Hi'].astype(str).str.replace(',','.')
df_clima_anteriores_sub['Temp Low'] = df_clima_anteriores_sub['Temp Low'].astype(str).str.replace(',','.')
df_clima_anteriores_sub['Humedad'] = df_clima_anteriores_sub['Humedad'].astype(str).str.replace(',','.')
df_clima_anteriores_sub['Radiacion'] = df_clima_anteriores_sub['Radiacion'].astype(str).str.replace(',','.')


# Transformado variables al formato correcto
# --
df_clima_anteriores_sub['FECHA'] =  pd.to_datetime(df_clima_anteriores_sub['FECHA'])
df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].astype('float')
df_clima_anteriores_sub['Temp Hi'] = df_clima_anteriores_sub['Temp Hi'].astype('float')
df_clima_anteriores_sub['Temp Low'] = df_clima_anteriores_sub['Temp Low'].astype('float')
df_clima_anteriores_sub['Humedad'] = df_clima_anteriores_sub['Humedad'].astype('float')
df_clima_anteriores_sub['Radiacion'] = df_clima_anteriores_sub['Radiacion'].astype('float')


C:\Users\CESAR QUEZADA\AppData\Local\Temp\ipykernel_19596\2654720774.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].replace('---',np.nan)
C:\Users\CESAR QUEZADA\AppData\Local\Temp\ipykernel_19596\2654720774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clima_anteriores_sub['Temperatura'] = df_clima_anteriores_sub['Temperatura'].replace('---',np.nan)
C:\Users\CESAR QUEZADA\AppData\Local\Temp\ipykernel_19596\2654720774.py:3: Set

In [85]:
df_clima_anteriores_sub.head()

,FECHA,Time,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion,PLANTACION
0,2010-01-01,00:30:00,20.9,21.0,20.8,100.0,0.0,Palmawasi
1,2010-01-01,01:00:00,20.8,20.9,20.8,100.0,0.0,Palmawasi
2,2010-01-01,01:30:00,20.8,20.8,20.7,100.0,0.0,Palmawasi
3,2010-01-01,02:00:00,20.8,20.8,20.8,100.0,0.0,Palmawasi
4,2010-01-01,02:30:00,20.8,20.9,20.8,100.0,0.0,Palmawasi


In [87]:
df_clima_full.head()

,FECHA,Time,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion,PLANTACION
0,2024-01-01,0:15,24.3,24.3,24.2,95.0,0.0,Nuevo Horizonte
1,2024-01-01,0:30,24.4,24.4,24.3,95.0,0.0,Nuevo Horizonte
2,2024-01-01,0:45,24.2,24.4,24.2,94.0,0.0,Nuevo Horizonte
3,2024-01-01,1:00,24.1,24.2,24.1,95.0,0.0,Nuevo Horizonte
4,2024-01-01,1:15,24.0,24.1,23.9,95.0,0.0,Nuevo Horizonte


In [88]:
# Uniendo los 2 dataframes
# ----
df_clima_full_2 = pd.concat([df_clima_anteriores_sub,
                             df_clima_full],
                             axis=0,
                             ignore_index=True)

In [91]:
df_clima_full_2.FECHA.max()

Timestamp('2024-12-06 00:00:00')

In [92]:
df_clima_full_vf = df_clima_full_2.groupby(['FECHA','PLANTACION']).agg({'Temperatura':'mean',
                                                                        'Temp Hi':'mean',
                                                                        'Temp Low':'mean',
                                                                        'Humedad':'mean',
                                                                        'Radiacion':'mean'}).reset_index()

df_clima_full_vf.head()

,FECHA,PLANTACION,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion
0,2010-01-01,Palmawasi,21.808511,21.982979,21.663830,94.914894,159.531915
1,2010-01-02,Palmawasi,21.793750,21.900000,21.658333,92.395833,144.916667
2,2010-01-03,Palmawasi,21.877083,22.033333,21.735417,87.000000,241.645833
3,2010-01-04,Palmawasi,22.202083,22.352083,22.047917,87.583333,178.395833
4,2010-01-05,Palmawasi,22.320833,22.493750,22.137500,89.395833,269.208333


#### Generando base total - Clima

In [96]:
# Guardamos la data en processed
# -----
# Especifica la ruta donde deseas guardar el CSV
ruta_clima_full_vf =  '/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_clima_full_vf.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_clima_full_vf.to_csv(ruta_path + ruta_clima_full_vf, index=False)
print(f'Datos guardados en {ruta_path + ruta_clima_full_vf}')

df_clima_full_vf.tail(10)

Datos guardados en C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_clima_full_vf.csv


,FECHA,PLANTACION,Temperatura,Temp Hi,Temp Low,Humedad,Radiacion
319893,2024-12-03,Shanuzi,26.663542,26.795833,26.527083,88.656250,186.385417
319894,2024-12-04,Nuevo Horizonte,28.053125,28.167708,27.931250,86.989583,221.156250
319895,2024-12-04,Palmawasi,27.242708,27.365625,27.106250,81.989583,206.260417
319896,2024-12-04,Shanuzi,27.913542,28.084375,27.745833,87.000000,188.447917
319897,2024-12-05,Nuevo Horizonte,28.156250,28.286458,28.019792,85.020833,213.531250
319898,2024-12-05,Palmawasi,27.221875,27.380208,27.063542,80.927083,228.208333
319899,2024-12-05,Shanuzi,27.892708,28.079167,27.726042,85.093750,233.250000
319900,2024-12-06,Nuevo Horizonte,25.866667,25.984375,25.733333,88.104167,167.343750
319901,2024-12-06,Palmawasi,25.506250,25.639583,25.380208,83.958333,169.677083
319902,2024-12-06,Shanuzi,26.059375,26.212500,25.910417,86.906250,201.510417
